# Load packages

In [1]:
import skvideo
skvideo.setFFmpegPath("/wecare/home/khoi/Downloads/ffmpeg-5.0.1-amd64-static")
import skvideo.io
import numpy as np
from pathlib import Path
import re
import os
import cv2
from os.path import join
from utils.data_path import *

# Ids range for demo

In [11]:
ids_range = range(100, 160)
IMG_DIR = join(AUGMENTED_SCANS, "fold_4")
MASK_DIR = join(AUGMENTED_MASKS, "fold_4")

filtered_ids = sorted([f for f in os.listdir(IMG_DIR) if re.search(f"CTP10_001_0[1-2][0-9][0-9].png" ,f)])
len(filtered_ids)

200

In [12]:
filtered_ids[:3]

['CTP10_001_0100.png', 'CTP10_001_0101.png', 'CTP10_001_0102.png']

# Create demo video

## Original video

In [14]:
vid_arr = np.zeros((200, 768, 768, 1))
for i, img_id in enumerate(filtered_ids):
    img_path = Path(IMG_DIR) / img_id
    #print(img_path)
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    vid_arr[i, ...] = img[:, :, np.newaxis]

In [15]:
skvideo.io.vwrite("demo/yolov5/original_v1.mp4", vid_arr)

## Video made from cropped patches

In [16]:
CROPS_DIR = join(YOLOV5_DETECT, "holdout_1/fold_4/crops/lesion/")

In [17]:
all_crops_id = sorted([f for f in os.listdir(CROPS_DIR) if re.search(f"CTP10_001_0[1-2][0-9][0-9][0-9]*.jpg" ,f)])
len(all_crops_id)

73

In [18]:
all_crops_id[18:25]

['CTP10_001_01342.jpg',
 'CTP10_001_0135.jpg',
 'CTP10_001_0136.jpg',
 'CTP10_001_0137.jpg',
 'CTP10_001_0138.jpg',
 'CTP10_001_0139.jpg',
 'CTP10_001_0140.jpg']

In [19]:
crops_arr = np.zeros((73, 192, 192, 1))
for i, img_id in enumerate(all_crops_id):
    img_path = Path(CROPS_DIR) / img_id
    #print(img_path)
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    crops_arr[i, ...] = img[:, :, np.newaxis]

In [20]:
skvideo.io.vwrite("demo/2D_Unet/crops_unsegmented_v1.mp4", crops_arr)

## Video made from cropped patches (segmented)

In [31]:
def process(mask_dir,  x):
    """
    Process the path of the original image. Take out the fold number and image ID. Then process it
    to create a proper path for 2D U-Net training, val or testing. Return a pair of image, one for
    cropped image and cropped mask.
    --------------------
    Parameters:
    
    mask_dir: Directory that contains all the cropped true masks. For example, /wecare/home/khoi/thesis/runs/yolov5/detect/holdout_1_true
    x: absolute path to the image
    """
    splits = tf.strings.split(x, sep = os.sep)
    fold, img_id = splits[-4], tf.strings.split(splits[-1], sep = ".")[0]
    img_path = x
    mask_path = tf.strings.join([tf.constant(mask_dir, dtype = tf.string),fold, "crops/lesion",img_id + '.png'], separator = os.sep)

    img = decode_png(read_file(img_path), channels = 1)
    mask = decode_png(read_file(mask_path), channels = 1)
    #img = tf.image.resize(img, [192, 192], method = "nearest")
    #mask = tf.image.resize(mask, [192, 192], method = "nearest")
    mask = tf.cast(equal(mask, 255), tf.float32)
    return img, mask

In [33]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.data import Dataset
from tensorflow.io import read_file, decode_png
os.environ["CUDA_VISIBLE_DEVICES"]="1"
holdout = 1

In [23]:
crops_abs_paths = sorted([join(CROPS_DIR, f) for f in os.listdir(CROPS_DIR) if re.search(f"CTP10_001_01[0-5][0-9][0-9]*.jpg" ,f)])
crops_abs_paths[:3]

['/wecare/home/khoi/thesis/runs/yolov5/detect/holdout_1/fold_4/crops/lesion/CTP10_001_0100.jpg',
 '/wecare/home/khoi/thesis/runs/yolov5/detect/holdout_1/fold_4/crops/lesion/CTP10_001_0101.jpg',
 '/wecare/home/khoi/thesis/runs/yolov5/detect/holdout_1/fold_4/crops/lesion/CTP10_001_0102.jpg']

In [27]:
demo_ds = Dataset.from_tensor_slices(crops_abs_paths)
len(demo_ds)

36

In [ ]:
demo_ds = demo_ds.map(lambda x: process(str(Path(YOLOV5_DETECT) / f"holdout_{holdout}_true"), x))

In [ ]:
model = 